<a href="https://colab.research.google.com/github/bieri2/ATMS-597-Project-4-Wx-Prediction/blob/master/ATMS597_GroupE_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import necessary modules
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

## KCMI Observations

In [0]:
# Read in daily KCMI data 
kcmi_daily = pd.read_csv('/content/drive/My Drive/project4_data/KCMI_daily.csv', 
                         header = 5, names = ['Timestamp', 'TMAXOBS', 'TMINOBS', 
                                              'WMAXOBS', 'RTOTOBS', 'extra'],
                         dtype = {'TMAXOBS': np.float64, 
                                  'TMINOBS': np.float64, 
                                  'WMAXOBS': np.float64, 
                                  'RTOTOBS': np.float64}, 
                         na_values = 'M')[:-7]
# Drop unnecessary column
kcmi_daily = kcmi_daily.drop(columns = 'extra')
# Set 'Date' column as index
kcmi_daily = kcmi_daily.set_index(pd.to_datetime(kcmi_daily['Timestamp'])).drop(columns = 'Timestamp')
# Resample to fill in missing days with NaNs
kcmi_daily = kcmi_daily.resample('D').mean()

In [0]:
# Read in hourly KCMI data
kcmi_hourly = pd.read_csv('/content/drive/My Drive/project4_data/KCMI_hourly.csv', 
                          header = 0, names = ['Timestamp', 'TMPC', 'DWPC', 'PRES', 
                                               'WDIR', 'WSPD', 'SKCT', 'PRCP1', 
                                               'PRCP6'], 
                          dtype = {'TMPC' : np.float64, 
                                   'DWPC' : np.float64, 
                                   'PRES' : np.float64, 
                                   'WDIR' : np.float64,
                                   'WSPD' : np.float64, 
                                   'SKCT' : np.float64, 
                                   'PRCP1': np.float64, 
                                   'PRCP6': np.float64}, 
                          usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8], na_values = 'M')
# Set 'Timestep' column as index
kcmi_hourly = kcmi_hourly.set_index(pd.to_datetime(kcmi_hourly['Timestamp'])).drop(columns = 'Timestamp')
# Consider only data for 2010 to 2019
kcmi_hourly = kcmi_hourly['2010-01-01':'2019-12-31']
# Set trace precipitation to NaN
kcmi_hourly['PRCP1'][kcmi_hourly['PRCP1'] < 0] = np.NaN

In [0]:
# Resample hourly data to daily by summing hourly values
kcmi_hourly_resampled = kcmi_hourly.resample('D').apply(lambda x: x.values.sum())

In [0]:
# Replace daily precip data with aggregated hourly data, since daily precip data is problematic
kcmi_daily['RTOTOBS'] = kcmi_hourly_resampled['PRCP1'].astype('float')

In [0]:
# Convert observations to metric units if necessary
kcmi_daily['TMAXOBS'] = (kcmi_daily['TMAXOBS'] - 32.)/1.8 # F to C
kcmi_daily['TMINOBS'] = (kcmi_daily['TMINOBS'] - 32.)/1.8 # F to C
kcmi_daily['WMAXOBS'] = kcmi_daily['WMAXOBS']/2.237 # mph to m/s

## GFS Data

In [0]:
# Code to decompress tar.gz files and add to directory
# Only need to do this once, afterwards just need to read from Drive
# Change paths/filenames as needed

# ! mkdir '/content/drive/My Drive/project4_data/sfc_tar'
# ! gunzip '/content/drive/My Drive/project4_data/sfc.tar.gz' 
# ! tar -xvf '/content/drive/My Drive/project4_data/sfc.tar' --directory '/content/drive/My Drive/project4_data/sfc_tar'

In [0]:
# Paths to files for different GFS datasets
# Change as needed
gfs_daily_dir = '/content/drive/My Drive/project4_data/daily_tar/bufkit/' # Daily GFS data
gfs_prof_dir  = '/content/drive/My Drive/project4_data/prof_tar/bufkit/'  # GFS 3-hr vertical profile 
gfs_sfc_dir   = '/content/drive/My Drive/project4_data/sfc_tar/bufkit/'   # GFS 3-hr surface data 

In [0]:
# Define function to read in GFS daily data one day at a time
# Once all files have been read, combine into one DataFrame 
def get_gfs_data(gfs_dir, prof = False, sfc = False):

  # Get list of files to be read
  files = os.listdir(gfs_dir)
  # Sort so that files are in order by date
  files.sort()
  # Create list of full paths for all files
  file_list = [gfs_dir + f for f in files]

  # Create empty list to hold data
  if prof:
      all_dwpc = []
      all_hght = []
      all_tmpc = []
      all_uwnd = []
      all_vwnd = [] 
  else:
      all_dfs  = []


  # Read in each file and add to list
  for f in file_list: 
      # Tell user which file is being read
      print('Reading ' + f)

      if prof:
          current = pd.read_csv(f, index_col = 0, header = 0,
                                names = ['Timestamp', 'DWPC', 'HGHT', 
                                         'PRES', 'TMPC', 'UWND', 'VWND'])  

          for i in current.index:
              dwpc_current = pd.DataFrame(current['DWPC'][i][1:-1].split(',')).T
              dwpc_current['Timestamp'] = [dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')]
              dwpc_current = dwpc_current.set_index('Timestamp')
              dwpc_current = dwpc_current.rename(columns = {0:'DWPC925', 1:'DWPC850', 2:'DWPC700', 
                                                            3:'DWPC500', 4:'DWPC250', 5:'DWPC100'})
              all_dwpc.append(dwpc_current) 

              hght_current = pd.DataFrame(current['HGHT'][i][1:-1].split(',')).T 
              hght_current['Timestamp'] = [dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')]
              hght_current = hght_current.set_index('Timestamp')
              hght_current = hght_current.rename(columns = {0:'HGHT925', 1:'HGHT850', 2:'HGHT700', 
                                                            3:'HGHT500', 4:'HGHT250', 5:'HGHT100'})
              all_hght.append(hght_current) 

              tmpc_current = pd.DataFrame(current['TMPC'][i][1:-1].split(',')).T 
              tmpc_current['Timestamp'] = [dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')]
              tmpc_current = tmpc_current.set_index('Timestamp')
              tmpc_current = tmpc_current.rename(columns = {0:'TMPC925', 1:'TMPC850', 2:'TMPC700', 
                                                            3:'TMPC500', 4:'TMPC250', 5:'TMPC100'})
              all_tmpc.append(tmpc_current) 

              uwnd_current = pd.DataFrame(current['UWND'][i][1:-1].split(',')).T 
              uwnd_current['Timestamp'] = [dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')]
              uwnd_current = uwnd_current.set_index('Timestamp')
              uwnd_current = uwnd_current.rename(columns = {0:'UWND925', 1:'UWND850', 2:'UWND700', 
                                                            3:'UWND500', 4:'UWND250', 5:'UWND100'})
              all_uwnd.append(uwnd_current) 

              vwnd_current = pd.DataFrame(current['VWND'][i][1:-1].split(',')).T 
              vwnd_current['Timestamp'] = [dt.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')]
              vwnd_current = vwnd_current.set_index('Timestamp')
              vwnd_current = vwnd_current.rename(columns = {0:'VWND925', 1:'VWND850', 2:'VWND700', 
                                                            3:'VWND500', 4:'VWND250', 5:'VWND100'})
              all_vwnd.append(vwnd_current) 
      elif sfc:
          current = pd.read_csv(f).T.reset_index()[1:].rename(columns = {'index': 'Timestamp', 0: 'DWPC', 
                                                                          1: 'HCLD', 2: 'LCLD', 3: 'MCLD', 
                                                                          4: 'PRCP', 5: 'PRES', 6: 'TMPC', 
                                                                          7: 'UWND', 8: 'VWND', 9: 'WSPD'})
          current = current.set_index('Timestamp')

          all_dfs.append(current)
      else:
          current = pd.read_csv(f)
          current = current.rename(columns = {'Unnamed: 0': 'Timestamp', 
                                                    'TMAX': 'TMAXGFS',
                                                    'TMIN': 'TMINGFS',
                                                    'WMAX': 'WMAXGFS',
                                                    'RTOT': 'RTOTGFS'})

          # Set timestamp as index
          current = current.set_index(pd.to_datetime(current['Timestamp'])).drop(columns = 'Timestamp')

          all_dfs.append(current) 

  if prof:
      all_dwpc = pd.concat(all_dwpc).astype(np.float64)
      all_hght = pd.concat(all_hght).astype(np.float64)
      all_tmpc = pd.concat(all_tmpc).astype(np.float64)
      all_uwnd = pd.concat(all_uwnd).astype(np.float64)
      all_vwnd = pd.concat(all_vwnd).astype(np.float64)

      all_prof = pd.concat([all_dwpc, all_hght, all_tmpc, 
                            all_uwnd, all_vwnd], axis = 1)
      
      return all_prof

  else: 
      # Create one DataFrame with all data 
      all_dfs = pd.concat(all_dfs).astype(np.float64)

      if not sfc:
          # Resample to fill any missing days
          all_dfs = all_dfs.resample('D').mean()

      return all_dfs

In [0]:
sfc = get_gfs_data(gfs_sfc_dir, sfc = True)

Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010112.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010212.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010312.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010412.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010512.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010612.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010712.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010812.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010010912.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/2010011012.gfs_kcmi.buf_sfc.csv
Reading /content/drive/My Drive/project4_data/sfc_tar/bufkit/201001111

In [0]:
# Read in profile GFS data
prof = get_gfs_data(gfs_prof_dir, prof = True)

Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010112.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010212.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010312.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010412.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010512.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010612.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010712.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010812.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010010912.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof_tar/bufkit/2010011012.gfs_kcmi.buf_prof.csv
Reading /content/drive/My Drive/project4_data/prof

In [0]:
sfc  = sfc.set_index(pd.to_datetime(sfc.index)).resample('3H').mean()
prof = prof.resample('3H').mean()
all_3hr_data = pd.concat([sfc['2010-01-01':'2019-12-31'], prof['2010-01-01':'2019-12-31']], axis = 1)

In [0]:
all_3hr_data['LCLD'][all_3hr_data['LCLD'] < 0] = np.NaN
all_3hr_data['MCLD'][all_3hr_data['MCLD'] < 0] = np.NaN
all_3hr_data['HCLD'][all_3hr_data['HCLD'] < 0] = np.NaN

In [0]:
all_3hr_data.to_csv('/content/drive/My Drive/ATMS 597 Project 4/all_data_3hr.csv')

In [0]:
# Read in daily GFS data
# This will take several minutes the first time but will be a lot faster if run again without restarting runtime
gfs_daily = get_gfs_data(gfs_daily_dir)

Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010112.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010212.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010312.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010412.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010512.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010612.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010712.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010812.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010010912.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Drive/project4_data/daily_tar/bufkit/2010011012.gfs_kcmi.buf_daily.csv
Reading /content/drive/My Driv

In [0]:
# Increase all dates in GFS data indices by one day since GFS forecasts apply to the following day
gfs_daily = gfs_daily.set_index(gfs_daily.index + pd.Timedelta(days = 1))

In [0]:
all_daily_data = pd.concat([kcmi_daily['2010-01-02':], gfs_daily[:'2019-12-31']], axis = 1)
all_daily_data.to_csv('/content/drive/My Drive/ATMS 597 Project 4/all_data_daily.csv')